# Difference Between `partitionBy` and `bucketBy` in Spark

## `partitionBy`
- **Operation Type**: Disk operation.
- **Description**: 
  - Divides data into separate directories based on the values of specified columns.
  - Each partition is written as a separate directory, which can enhance read performance by skipping irrelevant partitions during queries.
  
- **Advantages**:
  - **Improved Query Performance**: Queries that filter on the partitioned column can read only the relevant partitions, reducing I/O.
  - **Scalability**: Ideal for large datasets where certain columns are frequently queried.

- **When to Use**:
  - Use `partitionBy` when you frequently filter data based on specific columns (e.g., date, region).
  - Suitable for data that is large and can benefit from partitioned storage structure.

## `bucketBy`
- **Operation Type**: Disk operation.
- **Description**: 
  - Distributes data across a fixed number of buckets based on the hash of specified columns.
  - Each bucket is a separate file in the output directory.

- **Advantages**:
  - **Improved Joins**: When joining tables that are bucketed on the same columns and have the same number of buckets, Spark can perform efficient joins.
  - **Reduced Shuffle**: Helps in reducing shuffling when performing aggregations or joins on bucketed columns.

- **When to Use**:
  - Use `bucketBy` when you need to optimize joins or aggregations between large datasets.
  - Suitable for scenarios where data distribution across buckets can help enhance performance.

## Summary
- Use **`partitionBy`** for optimizing query performance based on filter conditions.
- Use **`bucketBy`** for optimizing join and aggregation operations between large datasets.


In [59]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder \
    .appName("SampleSparkCode") \
    .getOrCreate()

In [60]:
df_address = spark.read.csv("..\data\Address.csv",header=True)
df_customer = spark.read.csv("..\data\Customer.csv",header=True)
df_customer_address = spark.read.csv("..\data\CustomerAddress.csv",header=True)

In [61]:
df_dim_customer_address = df_customer.alias("c").join(
    df_customer_address.alias("ca"), col("c.CustomerID") ==  col("ca.CustomerID")) \
        .join(df_address.alias("a"),  col("ca.AddressID") ==  col("a.AddressID")) \
        .select(
     col("c.CustomerID").alias("CustomerID"),
     col("ca.AddressID").alias("AddressID"),
    "c.FirstName", "c.LastName", "c.CompanyName", "c.EmailAddress", "c.Phone",
    "ca.AddressType", "a.AddressLine1", "a.AddressLine2", "a.City", "a.StateProvince",
    "a.CountryRegion", "a.PostalCode"
        )

In [62]:
df_dim_customer_address.show(5)

+----------+---------+---------+-----------+--------------------+--------------------+------------+-----------+-----------------+------------+-----------+-------------+-------------+----------+
|CustomerID|AddressID|FirstName|   LastName|         CompanyName|        EmailAddress|       Phone|AddressType|     AddressLine1|AddressLine2|       City|StateProvince|CountryRegion|PostalCode|
+----------+---------+---------+-----------+--------------------+--------------------+------------+-----------+-----------------+------------+-----------+-------------+-------------+----------+
|     29485|     1086|Catherine|       Abel|Professional Sale...|catherine0@advent...|747-555-0171|Main Office|57251 Serene Blvd|        NULL|   Van Nuys|   California|United States|     91411|
|     29486|      621|      Kim|Abercrombie|      Riders Company|kim2@adventure-wo...|334-555-0137|Main Office|   Tanger Factory|        NULL|     Branch|    Minnesota|United States|     55056|
|     29489|     1069|  France

In [63]:
spark.sql("drop table if exists dim_customer_address")


DataFrame[]

- Partition by State
- Parition by can have both save and saveAsTable. In Save, it can take any relative path and save the files there

In [64]:
df_dim_customer_address.write.format("parquet") \
    .partitionBy("StateProvince") \
    .option("path","..\processed\example_partition_by\dimcustomer") \
    .mode("overwrite") \
    .saveAsTable("dim_customer_address")  

- Bucket By State( Note that Bucket does not allow save(). It allows only save as table)
- Based on the number given in bucketby, the files will be created (here 10 parquet files)
- Remember that bucketBy does not take relative path. It points to the warehouse directory - typically where spark is running. This is where Spark's default metastore resides

In [68]:
spark.sql("drop table if exists dim_customer_address")


DataFrame[]

In [69]:
df_dim_customer_address.write.format("parquet") \
    .bucketBy(10, "StateProvince") \
    .option("path", "..\processed\example_bucket_by\dimcustomer") \
    .mode("overwrite") \
    .saveAsTable("dim_customer_address")  


- The below is syntanctially correct, but it will not write to disk . Instead its in-memory only. Spark does not consider this partition by and still uses its own logical plan whenver this dataframe is filtered or queries or applied any transformation. So parition by or bucket by are useless unless its written to disk. 

In [67]:
df_dim_customer_address.write.format("parquet") \
    .partitionBy("StateProvince") 

- Writing this table without partition or bucket by saves the table as only one file

In [82]:
num_partitions = df_dim_customer_address.rdd.getNumPartitions()
print(num_partitions)

1


In [83]:
spark.sql("drop table if exists dim_customer_address")

DataFrame[]

In [84]:
df_dim_customer_address.write.format("parquet") \
    .option("path", "..\processed\plain_write\dimcustomer") \
    .mode("overwrite") \
    .saveAsTable("dim_customer_address") 

In [85]:
spark.stop()